In [ ]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

# Find the repository root by searching upward for a 'pyFMM' directory
start = Path.cwd().resolve()
repo_root = None
for p in [start] + list(start.parents):
    if (p / 'pyFMM').is_dir():
        repo_root = str(p)
        break
if repo_root is None:
    raise RuntimeError("Could not find 'pyFMM' in any parent directory of cwd")
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    
from pyFMM import *
import numpy as np

In [ ]:
p=8
#np.random.seed(42)


source_area_size =  1.0
LLC = np.array([ -1.0, -1.0, -1.0 ]) * source_area_size   # Lower Left Corner
URC = np.array([  1.0,  1.0,  1.0 ]) * source_area_size   # Upper Right Corner
size = URC - LLC

N_source = 1000
X = np.random.uniform(low=LLC, high=URC, size=(N_source, 3)) * 0.1

#X[:,2] = 0.0 # make it 2D in XY plane

X_sphe = cart_to_sphe(X) 
d = np.random.normal(size=(N_source, 3))
d = d / np.linalg.norm(d, axis=1)[:, np.newaxis]
q = np.random.uniform(low=-1.0, high=1.0, size=(N_source, ))

#------------- make target grid points - in XY plane to start with -------------
N_grid = 100
grid_points, X_grid, Y_grid = make_plane_grid(LLC, URC, N_grid, plane="xy")
#-------------------------------------------------------------------------------------
#--------------- Define rotation to arbitrary plane ----------------
theta = np.deg2rad(25.0)   # polar tilt away from XY
phi   = np.deg2rad(35.0)   # spin around z
R, u, v, n, center = rotate_plane(theta, phi, center=None)
#---------------------------------------------------------------------
#------------- rotate plane grid points ----------------
grid_points_rot = (R @ grid_points.T).T                     # apply rotation
grid_points_rot_sphe = cart_to_sphe(grid_points_rot)        # convert to spherical coordinates
#---------------------------------------------------------
#-------------- project source points onto plane for plotting --------------
X_proj = project_points_onto_plane(X, u, v, center=center)
#--------------------------------------------------------------------------

In [ ]:
#------------------- compute potentials directly ------------------------
P_dir = P_dipole_direct_cart(X, d, q, grid_points_rot, eps=1e-10)   # compute direct potential
P_dir_grid = P_dir.reshape(X_grid.shape[0], X_grid.shape[1])        # reshape to grid
#-----------------------------------------------------------------------

#--------------- compute potential via multipole moments ----------------
M = P2M_dip_sphe(X_sphe, d, q, p=p)                                 # compute multipole moments
P_mom = P_sphe(M, grid_points_rot_sphe)                             # compute potential via moments
P_mom_grid = P_mom.reshape(X_grid.shape[0], X_grid.shape[1])        # reshape to grid
#-----------------------------------------------------------------------

#------------------ define mask for plotting -----------------
min_plot_dist = 0.3 * source_area_size
plot_mask = np.sqrt(X_grid**2 + Y_grid**2) > min_plot_dist
#---------------------------------------------------------------

#------------ apply mask to potentials for plotting --------------
masked_P_dir_grid = np.where(plot_mask, P_dir_grid, 0)
masked_P_mom_grid = np.where(plot_mask, P_mom_grid, 0)
#---------------------------------------------------------------


In [ ]:
#-------------------- calculate differences ----------------------
difference_grid = masked_P_dir_grid - masked_P_mom_grid
rel_diff = np.zeros_like(difference_grid)
rel_diff[plot_mask] = difference_grid[plot_mask] / np.abs(masked_P_dir_grid[plot_mask]) * 100
#---------------------------------------------------------------


#--------------------- plotting ---------------------------------------------------------------------
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# First subplot: Direct potential with mask applied
plot_potential_grid(X_grid[0,:], Y_grid[:,0], masked_P_dir_grid.T, axes[0])
axes[0].scatter(X_proj[:, 0], X_proj[:, 1], color='k', s=5, label='Source Points')
axes[0].set_title("Direct Potential with Mask")
axes[0].set_xlabel("Plane axis 1")
axes[0].set_ylabel("Plane axis 2")
cbar = plt.colorbar(axes[0].collections[0], ax=axes[0])
cbar.set_label("Potential Value")

# Second subplot: Moment-based potential
plot_potential_grid(X_grid[0,:], Y_grid[:,0], masked_P_mom_grid.T, axes[1])
axes[1].set_title("Moment-based Potential")
axes[1].set_xlabel("Plane axis 1")
axes[1].set_ylabel("Plane axis 2")
cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar.set_label("Potential Value")

# Third subplot: Difference plot
plot_potential_grid(X_grid[0,:], Y_grid[:,0], difference_grid.T, axes[2])
axes[2].set_title("Difference (Direct - Moment)")
axes[2].set_xlabel("Plane axis 1")
axes[2].set_ylabel("Plane axis 2")

cbar = plt.colorbar(axes[2].collections[0], ax=axes[2])
cbar.set_label("Difference Value")

plt.tight_layout()
#----------------------------------------------------------------------------------------------------

In [ ]:
max_val = np.max(np.abs(masked_P_dir_grid))
max_diff = np.max(np.abs(difference_grid))
print(f"Max direct potential (masked): {max_val:.6e}")
print(f"Max difference between direct and moment-based potentials: {max_diff:.6e}")
print(f"Relative max difference: {max_diff / max_val * 100} %")

# Similar test but for monopoles

This test uses the same dipole points, but splits each into two monopoles of opposite charges seperated by a small distance in the 'd' direction.

In [ ]:
#-------------------- generate monopole pairs from dipoles ----------------------
X_mon, q_mon = dipoles_to_monopole_pairs(X,d,q)
X_mon_sphe = cart_to_sphe(X_mon)
#-------------------------------------------------------------------------------

In [ ]:
#------------------- compute potentials directly ------------------------
P_dir_mon = P_direct_cart(X_mon, q_mon, grid_points_rot, eps=1e-10)   # compute direct potential
P_dir_grid_mon = P_dir_mon.reshape(X_grid.shape[0], X_grid.shape[1])        # reshape to grid
#-----------------------------------------------------------------------
#--------------- compute potential via multipole moments ----------------
M_mon = P2M_sphe(X_mon_sphe, q_mon, p=p)                                 # compute multipole moments
P_mom_mon = P_sphe(M, grid_points_rot_sphe)                             # compute potential via moments
P_mom_grid_mon = P_mom_mon.reshape(X_grid.shape[0], X_grid.shape[1])        # reshape to grid
#-----------------------------------------------------------------------
#------------ apply mask to potentials for plotting --------------
# NOTE - uses same plot_mask as before
masked_P_dir_grid_mon = np.where(plot_mask, P_dir_grid_mon, 0)
masked_P_mom_grid_mon = np.where(plot_mask, P_mom_grid_mon, 0)
#---------------------------------------------------------------

In [ ]:
#-------------------- calculate differences ----------------------
difference_grid = masked_P_dir_grid_mon - masked_P_mom_grid_mon
rel_diff = np.zeros_like(difference_grid)
rel_diff[plot_mask] = difference_grid[plot_mask] / np.abs(masked_P_dir_grid[plot_mask]) * 100
#---------------------------------------------------------------


#--------------------- plotting ---------------------------------------------------------------------
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# First subplot: Direct potential with mask applied
plot_potential_grid(X_grid[0,:], Y_grid[:,0], masked_P_dir_grid_mon.T, axes[0])
axes[0].scatter(X_proj[:, 0], X_proj[:, 1], color='k', s=5, label='Source Points')
axes[0].set_title("Direct Potential with Mask")
axes[0].set_xlabel("Plane axis 1")
axes[0].set_ylabel("Plane axis 2")
cbar = plt.colorbar(axes[0].collections[0], ax=axes[0])
cbar.set_label("Potential Value")

# Second subplot: Moment-based potential
plot_potential_grid(X_grid[0,:], Y_grid[:,0], masked_P_mom_grid_mon.T, axes[1])
axes[1].set_title("Moment-based Potential")
axes[1].set_xlabel("Plane axis 1")
axes[1].set_ylabel("Plane axis 2")
cbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
cbar.set_label("Potential Value")

# Third subplot: Difference plot
plot_potential_grid(X_grid[0,:], Y_grid[:,0], difference_grid.T, axes[2])
axes[2].set_title("Difference (Direct - Moment)")
axes[2].set_xlabel("Plane axis 1")
axes[2].set_ylabel("Plane axis 2")

cbar = plt.colorbar(axes[2].collections[0], ax=axes[2])
cbar.set_label("Difference Value")

plt.tight_layout()
#----------------------------------------------------------------------------------------------------